In [55]:
import pandas as pd
import numpy as np
import h5py

import astropy.constants as const
import astropy.units as u
from astropy.coordinates import Galactocentric, ICRS, CartesianRepresentation, GeocentricTrueEcliptic
from astropy.coordinates import SkyCoord

In [56]:
hf_in = h5py.File('Galaxies/Galaxy1/galaxy_binaries.hdf5', 'r')

In [71]:
hf_in['sky']['dgb']['cat']['EclipticLatitude'][()]

array([-0.04256499, -1.01868065, -0.70346191, ..., -0.1423609 ,
        0.00520947,  0.43398823])

In [73]:
dat = pd.read_hdf('Galaxies/LISA_band_FZ_alpha25_Z.hdf')
print(dat.columns)

Index(['bin_num', 'mass_1', 'mass_2', 'kstar_1', 'kstar_2', 'sep', 'met',
       'tphys', 'rad_1', 'rad_2', 'xGx', 'yGx', 'zGx', 'FIRE_index', 'f_gw',
       'dist_sun'],
      dtype='object')


In [59]:
labels_gb = ['Amplitude', 'EclipticLatitude', 'EclipticLongitude', 'Frequency', 'FrequencyDerivative', 'Inclination', 'InitialPhase', 'Polarization']

In [117]:
def get_amplitude(dat):
    mc = (dat.mass_1.values*dat.mass_2.values)**(3/5) / (dat.mass_1.values + dat.mass_2.values)**(1/5) * u.Msun
    term1 = 64/5 * (const.G * mc)**(10/3)
    term2 = (np.pi*dat.f_gw.values*u.s**(-1))**(4/3)
    denom1 = const.c**8*(dat.dist_sun.values*u.kpc)**2
    amplitude = np.sqrt(term1.to(u.m**10/u.s**(20/3)) * term2 / denom1.to(u.m**10/u.s**8))
    return amplitude

def get_Gx_positions(dat):
    galcen = Galactocentric(x=dat.xGx.values*u.kpc, y=dat.yGx.values*u.kpc, z=dat.zGx.values*u.kpc)
    icrs = galcen.transform_to(ICRS())
    ecl = icrs.transform_to(GeocentricTrueEcliptic())
    return ecl.lon.to(u.rad), ecl.lat.to(u.rad)

def get_chirp(dat):
    mc = (dat.mass_1.values*dat.mass_2.values)**(3/5) / (dat.mass_1.values + dat.mass_2.values)**(1/5) * u.Msun
    fgw = dat.f_gw.values*u.s**(-1)
    term1 = (const.G * mc)**(5/3) / (const.c)**5
    term2 = (np.pi * fgw)**(11/3)
    chirp = 96/(5*np.pi) * term1 * term2
    return chirp.to(u.s**(-2))

def get_inc_phase_pol(dat):
    inc = np.arccos(np.random.uniform(0, 1, len(dat)))
    phase = np.random.uniform(0, np.pi, len(dat))
    pol = np.random.uniform(0, np.pi, len(dat))
    return inc, phase, pol

def create_dat_in(dat):
    h = get_amplitude(dat)
    lon, lat = get_Gx_positions(dat)
    chirp = get_chirp(dat)
    inc, phase, pol = get_inc_phase_pol(dat)
    dat_in = np.vstack([h, lon.value, lat.value, dat.f_gw.values, chirp.value, inc, phase, pol]).T

    return dat_in

In [5]:
import global_file_index as gfi

using mkl fft
mkl fft not available trying numpy


In [7]:
params_gb, n_dgb, n_igb, n_vgb, n_tot = gfi.get_full_galactic_params(
    galaxy_file='galaxy_binaries.hdf5', galaxy_dir='Galaxies/Galaxy1/', fmin=0.00001, fmax=0.1, 
    use_dgb=True, use_igb=False, use_vgb=False)

> /Users/kbreivik/Documents/Github/GalacticStochastic/global_file_index.py(55)get_full_galactic_params()
     53     import pdb
     54     pdb.set_trace()
---> 55     full_galactic_params_filename = get_galaxy_filename(galaxy_file, galaxy_dir)
     56     filename = full_galactic_params_filename
     57 



ipdb>  n


> /Users/kbreivik/Documents/Github/GalacticStochastic/global_file_index.py(56)get_full_galactic_params()
     54     pdb.set_trace()
     55     full_galactic_params_filename = get_galaxy_filename(galaxy_file, galaxy_dir)
---> 56     filename = full_galactic_params_filename
     57 
     58     hf_in = h5py.File(filename, 'r')



ipdb>  full_galactic_params_filename


'Galaxies/Galaxy1/galaxy_binaries.hdf5'


...

ipdb>  n


> /Users/kbreivik/Documents/Github/GalacticStochastic/global_file_index.py(89)get_full_galactic_params()
     87     params_gb = np.zeros((n_tot, n_par_gb))
     88     for itrl in range(0, n_par_gb):
---> 89         if use_dgb:
     90             params_gb[:n_dgb, itrl] = hf_in['sky']['dgb']['cat'][labels_gb[itrl]][mask_dgb]
     91         if use_igb:



ipdb>  labels_gb[itrl]


'Amplitude'


ipdb>  quit
